## Step1 导入相关包

In [1]:
import os

from langchain_community.llms import BaichuanLLM
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List
from sentence_transformers import CrossEncoder

d:\Download\miniconda3\envs\my_env\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
d:\Download\miniconda3\envs\my_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集 

In [2]:
data = "../WFs_reliability/"

In [ ]:
# 加载外部知识
loader = PyPDFDirectoryLoader(data)
docs_before_split = loader.load()


In [66]:
docs_before_split[15]

Document(metadata={'source': '..\\WFs_reliability\\风力发电设备可靠性评价规程（试行）.pdf', 'page': 15}, page_content=' 16附录 A \n（资料性附录）  \n风电设备可靠性统计状态中、英文对照表  \n中文  英文  缩写  \n在使用  active ACT \n可用  available A \n运行  in service S \n备用  reserve shutdown R \n调度停运备用  dispatching reserve shutdown hours DRH \n受累停运备用  passive reserve shutdown hours PRH \n场内原因受累停运备用  passive reserve shutdown inside hours PRIH \n场外原因受累停运备用  passive reserve shutdown outside \nhours PROH \n不可用  unavailable U \n计划停运  planned outage PO \n非计划停运  unplanned outage UO ')

In [69]:
# 过滤目录、附录和页码
docs= [doc.page_content.replace(str(doc.metadata['page']+1),'\n',1) for doc in docs_before_split if doc.metadata['page'] > 2 and doc.metadata['page'] <11] 

In [71]:
#文本分割
# 提取所有文本
full_text=""
for doc in docs:
    full_text += doc.replace("GAG",'').replace("INC",'')
print(full_text)

 
1  范围  
1.1 本规程规定了风力发电设备可靠性的统计办法和评价指标。 适
用于我国境内的所有风力发电企业。  
1.2 风力发电设备的可靠性统计评价包括风电机组的可靠性统计
评价和风电场的可靠性统计评价两部分。  
1.3 风电机组的可靠性统计评价范围以风电机组出口主开关为界，
包括风轮、传动变速系统、发电机系统、液压系统、偏航系统、控制
系统、变桨系统、通讯系统以及相应的辅助系统。  
1.4 风电场的可靠性统计评价范围包括风电场内的所有发电设备，
除了风电机组外，还包括箱变、汇流 线路、主变等，及其相应的附属、
辅助设备，公用系统和设施。  
2  基本要求  
2.1 本规程中指标评价所要求的各种基础数据报告，必须尊重科
学、事实求是、严肃认真、全面而客观地反映风力发电设备的真实情
况，做到准确、及时、完整。  
2.2 与本规程配套使用的“风电设备可靠性管理信息系统”软件及
相关代码，由中国电力企业联合会电力可靠性管理中心（以下简称“中
心”）组织编制，全国统一使用。  
3  状态划分  
风电机组（以下简称机组）状态划分如下： 
 
                     运行      
                      ( S )               
           可用 ( A )              
                           调度停运备用         
                     备用       ( D R )                  
( R )                            场内原因受累停运备用  
在使用                   受累停运备用          ( P R I )      
（ACT）                    ( P R )        场外原因受累停运备用         
                                               ( P R O )     
                       计划停运               
           不可用 (U)     (PO)  
             

In [96]:
import re
docs = re.split(r"\d\x20{2}",full_text)
docs.pop(0)
print(docs[8])

风电场非计划停运系数（ UOFs） 
计算公式为：  
UOFs＝［                                  ］× 100% 
 
8.


In [90]:
# 将分割后的文本块封装成Document类
docs_after_split = []
for i in range(0,len(docs)):
    qa_doc = Document(page_content=docs[i], metadata={"source": "../WFs_reliability/风力发电设备可靠性评价规程（试行）.pdf","order":i+1})
    docs_after_split.append(qa_doc)

docs_after_split[3]

Document(metadata={'source': '../WFs_reliability/风力发电设备可靠性评价规程（试行）.pdf', 'order': 4}, page_content='术语和定义  \n4.1 在使用（ ACT）——机组处于要进行统计评价的状态。在使用\n状态分为可用（ A）和不可用（ U）。 \n4.2 可用（ A）——机组处于能够执行预定功能的状态，而不论其\n是否在运行，也不论其提供了多少出力。可用状态分为运行（ S）和备\n用（ R）。 \n4.2.1 运行（ S）——机组在电气上处于联接到电力系统的状态，\n或虽未联接到电力系统但在风速条件满足时，可以自动联接到电力系\n统的状态。机组在运行状态时，可以是带出力运行，也可以是因风速过低没有出力。  \n4.2.2 备用（ R）——机组处于可用，但不在运行状态。备用可分\n为调度停运备用（ DR）和受累停运备用（ PR）。 \n4.2.2.1 调度停运备用（ DR）——机组本身可用，但因电力系统需\n要，执行调度命令的停运状态。  \n4.2.2.2 受累停运备用（ PR）——机组本身可用，因机组以外原因\n造成的机组被迫退出运行的状态。按引起受累停运的原因，可分为场\n内原因受累停运备用（ PRI）和场外原因受累停运备用（ PRO）。 \na) 场内原因受累停运备用（ PRI）——因机组以外的场内设备停运\n（如汇流线路、箱变、主变等故障或计划检修）造成机组被迫退出运行的状态。  \nb) 场外原因受累停运备用（ PRO）——因场外原因（如外部输电\n线路、电力系统故障等）造成机组被迫退出运行的状态。  \n4.3 不可用（ U）——机组不论什么原因处于不能运行或备用的状\n态。不可用状态分为计划停运（ PO）和非计划停运（ UO）。 \n4.3.1计划停运（ PO）——机组处于计划检修或维护的状态。计划\n停运应是事先安排好进度，并有既定期限的定期维护。  \n4.3.2非计划停运 （UO）——机组不可用而又不是计划停运的状态。  \n4.4额定容量（ \n）——指一机组的铭牌额定容量（ ）。 \n4.5实际发电量 （）——指机组在给定期间内实际发出的电量。  \n4.6运行小时（ SH）——机组处于运行状态的小时数。   \n4.7备用小时（ RH）——机组处于备用状态的

In [103]:
# 从过往工作经验看，embedding对于rag效果影响比较大，一般首选还是openai embedding做这一部分，开源的效果很一般
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 使用m3e模型做embeddding
    model_kwargs={'device':0},
    encode_kwargs={'normalize_embeddings': True}
)

In [104]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

In [105]:
query = """风电场有关的术语和定义"""
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query,k=10)
for i, doc in enumerate(relevant_documents):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


检索到的第1个内容: 
 风电场评价指标  
8.1 风电场评价指标按机组指标的容量加权平均值进行计算。  
8.2 当统计风电场指标时，把因场内原因受累停运备用状态（ PRI）
的机组视为不可用，其受累停运备用小时（ PRIH）计入不可用小时。
这时的机组可用小时（ AH 1）等于运行小时、调度停运备用小时和场外
原因受累停运备用小时之和。用公式表示：  
AH 1=SH＋DRH＋PROH 
8.3 风电场可用系数  (AFs) 
计算公式为：  
 AFs＝［                    ］× 100% 
 
8.
-----------------------------------------------------
检索到的第2个内容: 
 风电场利用小时（ UTHF） 
计算公式为：  ∑（AH 1×GMC） 
∑(GMC×PH) 
∑((UOH+ PRIH ) ×GMC) 
∑(GMC ×PH)  
UTHF＝        
 
 9 基础数据注册  
9.1 所有机组均应按规定代码、编号进行注册。  
9.1 机组注册内容、 机组主设备注册内容按表 1～2要求进行填报。  
10 事件数据填写规定  
10.1事件代码是描述设备故障及其原因的特殊标识符，是基础数
据的重要组成部分，所有代码应遵循“中心”对风电设备的有关要求填写。机组的所有计划和非计划或受累停运备用事件，都应填写相应的事件代码。  
10.2 跨月事件必须拆成两条记录，迄于上月末记录和始于下月初
记录。两条记录必须保持时间连续、状态、代码等一致。  
10.3 机组计划检修以及非计划检修事件， 应填写检修工日和费用。  
10.4 当机组发生非计划停运或受累停运备用时，除了要填写事件
代码外，还应填写电量损失值（ EL）。 
11 统计评价报告  
11.1 可靠性基础数据报告，分为四种（表 1～4）：即机组注册内
容报表、机组主设备注册内容、机组月度发电量报表、机组月度事件数据报表。  
11.2 机组可靠性基础数据由发电企业记录和统计，并按《电力可
靠性监督管理办法》规定的报送时间和审核程序上报。  
11.3 报告若需修改，必须以文件形式逐级上报，说明更改内容和
变更原因；各级主管部门对上报的报告必须认真核实后进行转报；修改已报出“基础数据”须下次报告时一并完成。  

In [106]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(
    "BAAI/bge-reranker-base", max_length=512, device=0
)
reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=2,
        return_documents=True,)
reranked_docs

[{'corpus_id': 6,
  'score': 0.4401685,
  'text': '术语和定义  \n4.1 在使用（ ACT）——机组处于要进行统计评价的状态。在使用\n状态分为可用（ A）和不可用（ U）。 \n4.2 可用（ A）——机组处于能够执行预定功能的状态，而不论其\n是否在运行，也不论其提供了多少出力。可用状态分为运行（ S）和备\n用（ R）。 \n4.2.1 运行（ S）——机组在电气上处于联接到电力系统的状态，\n或虽未联接到电力系统但在风速条件满足时，可以自动联接到电力系\n统的状态。机组在运行状态时，可以是带出力运行，也可以是因风速过低没有出力。  \n4.2.2 备用（ R）——机组处于可用，但不在运行状态。备用可分\n为调度停运备用（ DR）和受累停运备用（ PR）。 \n4.2.2.1 调度停运备用（ DR）——机组本身可用，但因电力系统需\n要，执行调度命令的停运状态。  \n4.2.2.2 受累停运备用（ PR）——机组本身可用，因机组以外原因\n造成的机组被迫退出运行的状态。按引起受累停运的原因，可分为场\n内原因受累停运备用（ PRI）和场外原因受累停运备用（ PRO）。 \na) 场内原因受累停运备用（ PRI）——因机组以外的场内设备停运\n（如汇流线路、箱变、主变等故障或计划检修）造成机组被迫退出运行的状态。  \nb) 场外原因受累停运备用（ PRO）——因场外原因（如外部输电\n线路、电力系统故障等）造成机组被迫退出运行的状态。  \n4.3 不可用（ U）——机组不论什么原因处于不能运行或备用的状\n态。不可用状态分为计划停运（ PO）和非计划停运（ UO）。 \n4.3.1计划停运（ PO）——机组处于计划检修或维护的状态。计划\n停运应是事先安排好进度，并有既定期限的定期维护。  \n4.3.2非计划停运 （UO）——机组不可用而又不是计划停运的状态。  \n4.4额定容量（ \n）——指一机组的铭牌额定容量（ ）。 \n4.5实际发电量 （）——指机组在给定期间内实际发出的电量。  \n4.6运行小时（ SH）——机组处于运行状态的小时数。   \n4.7备用小时（ RH）——机组处于备用状态的小时数。  \n用公式可表示为：  \nRH=DRH+PRH ＝DRH+ PRIH ＋PROH，其

In [113]:
""" 使用自定义的检索器,重写_get_relevant_documents方法来实现对相似性检索召回的文本块进行重排"""
class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        cross_encoder = CrossEncoder("BAAI/bge-reranker-base", max_length=512, device=0)
        relevant_documents = vectorstore.similarity_search(query,k=10)
        reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=2,
        return_documents=True,)

        reranking_relevant_documents= []
        for doc in reranked_docs:
           reranking_relevant_documents.append(
               Document(page_content=doc["text"],
                        metadata=relevant_documents[doc["corpus_id"]].metadata)
                        ) 
        return reranking_relevant_documents

In [114]:
# 自定义加入重排的检索器
retriever = CustomRetriever()

In [109]:
# 创建prompt
prompt_template = """
请参考下面的相关文本回答问题，一段文本中可能有多个子问题，子问题以问号“？”分隔，子问题对应的答案以小括号“（）”进行分隔，
检索到与问题相关的子问题直接输出子问题对应的答案的全部内容，如果不知道答案，就回复不知道，

{context}

Question: {question}

Helpful Answer:
"""

prompt = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [110]:
# 创建百川的LLM
llm = BaichuanLLM(model='Baichuan3-Turbo-128k')

In [115]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [116]:
question = "风电场有关的术语和定义"
result = qa_chain({"query": question})
# result["result"]
print("========= chain result ==========")
print(result['result'])

========= chain result ==========
风电场有关的术语和定义包括：

4.1 在使用（ACT）——机组处于要进行统计评价的状态。在使用状态分为可用（A）和不可用（U）。
4.2 可用（A）——机组处于能够执行预定功能的状态，而不论其是否在运行，也不论其提供了多少出力。可用状态分为运行（S）和备用（R）。
4.2.1 运行（S）——机组在电气上处于联接到电力系统的状态，或虽未联接到电力系统但在风速条件满足时，可以自动联接到电力系统的状态。机组在运行状态时，可以是带出力运行，也可以是因风速过低没有出力。
4.2.2 备用（R）——机组处于可用，但不在运行状态。备用可分为调度停运备用（DR）和受累停运备用（PR）。
4.2.2.1 调度停运备用（DR）——机组本身可用，但因电力系统需要，执行调度命令的停运状态。
4.2.2.2 受累停运备用（PR）——机组本身可用，因机组以外原因造成的机组被迫退出运行的状态。按引起受累停运的原因，可分为场内原因受累停运备用（PRI）和场外原因受累停运备用（PRO）。
4.3 不可用（U）——机组不论什么原因处于不能运行或备用的状态。不可用状态分为计划停运（PO）和非计划停运（UO）。
4.3.1 计划停运（PO）——机组处于计划检修或维护的状态。计划停运应是事先安排好进度，并有既定期限的定期维护。
4.3.2 非计划停运（UO）——机组不可用而又不是计划停运的状态。
4.4 额定容量（）——指一机组的铭牌额定容量（）。
4.5 实际发电量（）——指机组在给定期间内实际发出的电量。
4.6 运行小时（SH）——机组处于运行状态的小时数。
4.7 备用小时（RH）——机组处于备用状态的小时数。
4.7.1 调度停运备用小时（DRH）——机组处于调度停运备用状态的小时数。
4.7.2 受累停运备用小时（PRH）——机组处于受累停运备用状态的小时数。受累停运备用小时又可分为下列2类：
a) 场内原因受累停运备用小时数（PRIH）——机组处于场内原因受累停运备用状态的小时数。
b) 场外原因受累停运备用小时数（PROH）——机组处于场外原因受累停运备用状态的小时数。
4.8 计划停运小时（POH）——机组处于计划停运状态的小时数。
4.9 非计划停运小时（UOH）——机组处于非计划停运状态的小时数。
4.10 统计期间